In [1]:
import pandas as pd
import boto3
import sagemaker
comprehend = boto3.client('comprehend', region_name='us-east-1')

In [2]:
bucket = sagemaker.Session().default_bucket()
print('Bucket:\n{}'.format(bucket))

Bucket:
sagemaker-us-east-1-737786887809


In [3]:
data_bucket = 'nlp-otus-homework'
lang_data_filename = 'lang_data.csv'
s3 = boto3.resource('s3')
s3.Bucket(data_bucket).download_file(lang_data_filename, 'raw_data.csv')

data = pd.read_csv('./raw_data.csv')
data.head()

,text,language
0,Ship shape and Bristol fashion,English
1,Know the ropes,English
2,Graveyard shift,English
3,Milk of human kindness,English
4,Touch with a barge-pole - Wouldn't,English


In [20]:
data.isnull().values.any()

False

In [19]:
data.dropna(inplace=True)

In [22]:
data.language.value_counts()

English       2055
Afrikaans      639
Nederlands      67
Name: language, dtype: int64

In [5]:
lang = comprehend.detect_dominant_language(Text=data.text[0])

In [6]:
lang

{'Languages': [{'LanguageCode': 'en', 'Score': 0.9807940721511841}],
 'ResponseMetadata': {'RequestId': '5a51c77f-82a9-47d2-a802-1ef5e26d358e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5a51c77f-82a9-47d2-a802-1ef5e26d358e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '64',
   'date': 'Mon, 27 Jul 2020 09:25:19 GMT'},
  'RetryAttempts': 0}}

In [24]:
for index, row in data.iterrows():         
    data.loc[index,'detected_language'] = comprehend.detect_dominant_language(Text=row.text)['Languages'][0]['LanguageCode']

In [25]:
data.head()

,text,language,detected_language
0,Ship shape and Bristol fashion,English,en
1,Know the ropes,English,en
2,Graveyard shift,English,en
3,Milk of human kindness,English,en
4,Touch with a barge-pole - Wouldn't,English,en


In [26]:
data.language.value_counts()

English       2055
Afrikaans      639
Nederlands      67
Name: language, dtype: int64

In [27]:
data.detected_language.value_counts()

en    2038
af     529
nl     158
fr      13
es       6
la       4
da       2
it       2
uz       2
de       2
pt       1
lt       1
lv       1
ca       1
id       1
Name: detected_language, dtype: int64

In [28]:
# Function to upload to S3
from io import StringIO
import boto3

def write_pd_s3_csv(df, bucket, filepath):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, filepath).put(Body=csv_buffer.getvalue())
    print("The data is successfully written to S3 path:", bucket+"/"+filepath)

# Write to S3
file_path = 'detected_data.csv'
write_pd_s3_csv(data, data_bucket, file_path)

The data is successfully written to S3 path: nlp-otus-homework/detected_data.csv


Посмотрим долю правильно опознанных английских фраз

In [34]:
len(data[(data.language == 'English') & (data.detected_language == 'en')].index) / len(data[data.language == 'English'].index)

0.9854014598540146

Доля правильно опознанных голландских фраз:

In [36]:
len(data[(data.language == 'Nederlands') & (data.detected_language == 'nl')].index) / len(data[data.language == 'Nederlands'].index)

1.0

Доля правильно опознанных фраз на языке африкаанс:

In [37]:
len(data[(data.language == 'Afrikaans') & (data.detected_language == 'af')].index) / len(data[data.language == 'Afrikaans'].index)

0.8278560250391236

Посмотрим, с какими языками Comprehend перепутал английский

In [38]:
data[(data.language == 'English') & (data.detected_language != 'en')]

,text,language,detected_language
109,Hoi polloi,English,nl
118,Deus ex machina,English,pt
229,Aide-mémoire,English,fr
240,La dolce vita,English,it
282,Savoir faire,English,fr
335,"Et tu, Brute",English,fr
351,Carte blanche,English,fr
391,Cordon bleu,English,fr
424,"Hasta la vista, baby",English,es
569,Fait accompli,English,fr


Здесь видны заимствования из французского, латыни, итальянского и пр., поэтому ошибки Comprehend вполне обоснованы.

Посмотрим, с какими языками Comprehend перепутал африкаанс

In [40]:
data[(data.language == 'Afrikaans') & (data.detected_language != 'af')]

,text,language,detected_language
35,"Vroeg uit die bed, maak die beursie vet!",Afrikaans,nl
54,Kop bo water hou.,Afrikaans,nl
135,Soebat en bangoog kyk help niks.,Afrikaans,id
146,Met ‘n aalwynpil in jou kies loop.,Afrikaans,nl
177,Geen spiesen maacht so diepe wonden als achter...,Afrikaans,nl
...,...,...,...
2711,Die geheim om te verveel is om alles te vertel.,Afrikaans,nl
2727,Iemand afpoeier.,Afrikaans,nl
2755,Een hoofd vol kreuken – een geweten sonder rim...,Afrikaans,nl
2761,Johann Nell: “Dit is baie beter om te huil in ...,Afrikaans,nl


In [42]:
data[(data.language == 'Afrikaans') & (data.detected_language != 'af')]['detected_language'].value_counts()

nl    89
en    13
uz     2
da     2
fr     1
de     1
id     1
lt     1
Name: detected_language, dtype: int64

Как и ожидалось, Comprehend чаще всего путает африкаанс с голландским, от которого африкаанс и произошел.